In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 11 01:08:27 2016

@author: pehqincheng
"""
## Need to label graph by subzone also
import networkx as nx
import matplotlib.pyplot as plt
import random 
import copy
import csv
import math

def combine_graph(graph_array):
    anchor_graph = graph_array.pop(0)
    for i in graph_array:
        ## Create start end index for range
        start = len(anchor_graph)
        end = start + len(i)
        mapping=dict(zip(i.nodes(),range(start,end)))
        ## make new graph that is relabelled
        rlb_graph = nx.relabel_nodes(i,mapping)
        ## Add relabelled graph into anchor graph
        anchor_graph.add_nodes_from(rlb_graph.nodes(data=True))
        anchor_graph.add_edges_from(rlb_graph.edges(data=True))
    return anchor_graph

def read_csv(csv_path):
    row_con = []
    loc_count = {}
    loc_coord = {}
    pop = {}
    with open(csv_path, 'rb') as f:
        reader = csv.reader(f)
        reader.next()
        for row in reader:
            pop[row[1]] = int(row[9])
            loc_count[row[1]] = 0
            loc_coord[row[1]] = (float(row[11].replace(",","")), float(row[12].replace(",","")))
            con = {'location':row[1],'zone':row[3],'nodes_num':int(row[9]),'prob':float(row[10])}           
            row_con.append(con)
    return row_con,loc_count,loc_coord,pop

def flip(p):
    return 'I' if random.random() <= p else 'S'
    
#er=nx.erdos_renyi_graph(10,0.5)
#erc = copy.deepcopy(er)
#er.node[1]['status'] = 'I'
#print er.node[1]
#
#er2 = nx.erdos_renyi_graph(10,0.5)
#er2c = copy.deepcopy(er2)
#
#
#
#er3 = nx.erdos_renyi_graph(20,0.3)
#new_graph = combine_graph([erc,er2c])
#nx.draw(new_graph)
#plt.show()

perc = 5
inf_prob = 0.4
dict_con,loc_count,loc_coord, pop = read_csv('population2.csv')

g_con = []
for a in dict_con:
    g = nx.erdos_renyi_graph(a['nodes_num'],a['prob']/10.0)
    nx.set_node_attributes(g,'zone',a['zone'])
    nx.set_node_attributes(g,'location',a['location'])
    g_con.append(g)

f_graph = combine_graph(g_con)
for i in f_graph.nodes():
    f_graph.node[i]['status'] = 'S'

total_pop = nx.number_of_nodes(f_graph)
# infected_no = int(round(total_pop*(perc/100.0)))


In [ ]:
print nx.number_of_edges(f_graph)

In [ ]:
neighbors = {}
distances = []
for location,coord in loc_coord.iteritems():
    gravities = []
    if pop[location] == 0:
        continue
    for key,value in loc_coord.iteritems():
        dist = math.sqrt( (coord[0] - value[0])**2 + (coord[1] - value[1])**2 )
        if dist == 0 or pop[key] == 0:
            continue
        G = float(pop[location]*pop[key])/dist**2
        gravities.append([G,key])
    gravities = sorted(gravities,key=lambda x: x[0],reverse=True)
    for i in gravities[:4]:
        distances.append(i[0])
    neighbors[location] = gravities[:4]
    
distances.sort()
print len(distances)

    
#myFile= open( 'out.csv', 'wb' )
#wtr= csv.writer( myFile )
#with open('population.csv', 'rb') as f:
#    reader = csv.reader(f)
#    header = reader.next()
#    wtr.writerow(header + ['Inf_pop'])
#    for row in reader:
#        wtr.writerow(row + [loc[row[1]]])
#myFile.close()

In [ ]:
ranges = []
for i in range(1,6):
    if i == 5:    
        ranges.append((distances[(i-1)*190],distances[-1]))
    else:
        ranges.append((distances[(i-1)*190],distances[(i)*190]))
print ranges


In [ ]:
def bin_pop(value,ranges):
    for i in ranges:
        if i[0]<= value < i[1]:
            return 0.001 + 0.003*ranges.index(i)
    print value
    return 0.001 + 0.003*4
for key,n in neighbors.iteritems():
    for i in n:
        i.append(bin_pop(i[0],ranges))
print neighbors

In [ ]:
def make_edge(p):
    return True if random.random() <= p else False

for pa,n in neighbors.iteritems():
    cluster1 = list((n for n in f_graph if f_graph.node[n]['location']==pa))
    for i in n:
        cluster2 = list((n for n in f_graph if f_graph.node[n]['location']==i[1]))
        for j in cluster1:
            for k in cluster2:
                if make_edge(i[2]):
                    f_graph.add_edge(j,k)


In [ ]:
print nx.number_of_edges(f_graph)
a = list((n for n in f_graph if f_graph.node[n]['location']=='Tampines East'))
print a[0], a[-1]

In [ ]:
infected_no = 3

infected_seed = set(random.sample(range(28634,30019),infected_no))
for i in infected_seed:
    f_graph.node[i]['status'] = 'I'
for a in range(20):
    for i in infected_seed:
        for n in f_graph.neighbors(i):
            if f_graph.node[n]['status'] != 'I':
                f_graph.node[n]['status'] = flip(inf_prob)
                
    for i,dict in f_graph.nodes_iter(data=True):
        if dict['status'] == 'I':  
            loc_count[dict['location']] += 1
            infected_seed.add(i)        
    print loc_count
    filename = "out" + str(a) +'.csv'
    myFile= open("out" + str(a+1) +'.csv', 'wb' )
    wtr= csv.writer( myFile )
    with open(filename, 'rb') as f:
       reader = csv.reader(f)
       header = reader.next()
       wtr.writerow(header + ['Inf_pop day ' + str(a)])
       for row in reader:
           wtr.writerow(row + [loc_count[row[1]]])
    myFile.close()
    
    for key,value in loc_count.iteritems():
        value = 0
    print loc_count